In [4]:
import cv2
import numpy as np
import csv
import matplotlib.pyplot as plt
import statistics as stat

In [5]:
filename = "video4.avi"
location = "E:/Sawani/data/210318/F1 control block 1/"

x_coor = []
y_coor = []

count = 0

cap = cv2.VideoCapture(location + filename)

while cap.isOpened:
    
    ret,frame = cap.read()
    
    if ret == 0:
        break
    
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(frame, (5, 5), 0)
    thresh = cv2.adaptiveThreshold(blur, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY_INV,25,15) 
    
    kernel = np.ones((3,3),np.uint8)
    erode = cv2.erode(thresh, kernel, iterations = 1)
    kernel = np.ones((3,3),np.uint8)
    dilate = cv2.dilate(erode, kernel, iterations = 5)
    
    _, contours, _ = cv2.findContours(dilate, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    largest = max(contours, key = cv2.contourArea)
    
    (x,y),radius = cv2.minEnclosingCircle(largest)
    center = (int(x),int(y))
    radius = int(radius)
    cv2.circle(frame, center, 5, 5)
    x_coor.append(int(x))
    y_coor.append(int(y))
    
    #for i in range(len(x_coor) - 1):
        #cv2.circle(frame, (x_coor[i], y_coor[i]), 2, -1)
    
    
    cv2.imshow("frame", frame)  
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    count += 1
    
cap.release()
cv2.destroyAllWindows()

In [ ]:
plt.figure(figsize = (10, 10))
plt.scatter(x_coor, y_coor)

output = [x_coor, y_coor]
fields = ["x","y"]

with open(location + filename[:-4] + "coordinates.csv", "w") as csv_file:
    csvwriter = csv.writer(csv_file)
    csvwriter.writerow(fields)
    for i in range(len(output[0])):
        csvwriter.writerow([output[0][i], output[1][i]])

In [ ]:
dist = []
vel = []

for i in range(len(x_coor) - 1):
    distance = ((x_coor[i + 1] - x_coor[i])**2 + (y_coor[i + 1] - y_coor[i])**2)**0.5
    if distance <= 4:
        dist.append(0)
    else:
        dist.append(distance)
    #if i != 0:
        #dist[-1] += dist[-2]

In [ ]:
plt.figure(figsize = (20, 10))
plt.plot(np.arange(0, 10, 0.01), dist[:1000])
plt.xlabel("time (s)", fontsize = 15)
plt.ylabel("distance / frame (pixels)", fontsize = 15)
plt.xlim(5, 6.5)
plt.savefig("./dist_time_plot.png")

In [ ]:
plt.figure(figsize = (20, 10))
plt.plot(dist)
plt.xlabel("frames", fontsize = 15)
plt.ylabel("distance / frame (pixels)", fontsize = 15)
#plt.xlim(5, 5.25)
plt.savefig("./dist_time_plot.png")